In [ ]:
# ---
# ETAPA DE REQUISIÇÃO E EXTRAÇÃO DE DADOS DA API
# ---

import requests
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

table_name = "default.creators_scrape_wiki"

df_wiki_pages = spark.read.table(table_name)

# Exibe o schema e os dados para verificar se a leitura funcionou
df_wiki_pages.printSchema()
df_wiki_pages.show(5, False)



def get_yt_user_from_wiki(page_name):
    # Endpoint da API da Wikipedia
    url = "https://en.wikipedia.org/w/api.php"
    
    # Parâmetros da requisição para extrair o HTML
    params_html = {
        "action": "parse",
        "page": page_name,
        "prop": "text",
        "format": "json"
    }

    # Parâmetros da requisição para extrair os sitelinks (wikidata)
    params_sitelinks = {
        "action": "wbgetentities",
        "sites": "enwiki",
        "titles": page_name,
        "props": "sitelinks",
        "format": "json"
    }
    
    try:
        # Fazendo a busca por sitelinks na Wikidata
        response_sitelinks = requests.get(url, params=params_sitelinks)
        response_sitelinks.raise_for_status()
        data_sitelinks = response_sitelinks.json()
        if "entities" in data_sitelinks:
            for entity_id in data_sitelinks["entities"]:
                entity = data_sitelinks["entities"][entity_id]
                if "sitelinks" in entity:
                    # Verifica se há link para o YouTube
                    if "enwikiquote" in entity["sitelinks"] and "yt_user" in entity["sitelinks"]["enwikiquote"]:
                        return entity["sitelinks"]["enwikiquote"]["yt_user"]

        # Se a busca por sitelinks falhar, volta para a busca no HTML
        response_html = requests.get(url, params=params_html)
        response_html.raise_for_status()
        data_html = response_html.json()
        
        # Extração do HTML do conteúdo da página
        html_content = data_html['parse']['text']['*']
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Expressão regular para capturar IDs de usuário, canal e o novo formato com '@'
        regex = re.compile(r'https?://(?:www\.)?youtube\.com/(?:channel/|user/|c/|@)?([a-zA-Z0-9_-]+)')
        
        # Busca nos links dentro de infoboxes
        infobox = soup.find('table', class_='infobox')
        if infobox:
            for link in infobox.find_all('a', href=True):
                match = regex.search(link['href'])
                if match:
                    return match.group(1)

        # Busca na seção de links externos
        external_links = soup.find(id='External_links')
        if external_links:
            for link in external_links.find_next('ul').find_all('a', href=True):
                match = regex.search(link['href'])
                if match:
                    return match.group(1)
        return None
        
    except Exception as e:
        print(f"Erro ao processar a página {page_name}: {e}")
        return None

get_yt_user_udf = udf(get_yt_user_from_wiki, StringType())
# Aplica a função no DataFrame para criar a coluna 'user_id'
df_users_yt = df_wiki_pages.withColumn(
    "user_id",
    get_yt_user_udf(col("wiki_page"))
).select("user_id", "wiki_page")


df_users_yt.show(5, False)

# Salva o DataFrame como uma tabela Delta
df_users_yt.write.format("delta").mode("overwrite").saveAsTable("default.users_yt")

print("Tabela 'default.users_yt' criada com sucesso!")